# Modeling d0eop_microkappa
Using the feautures of the stage p predict the blancura salida etapa P - blancura en línea

-------
**INFO MODEL**
- **stage**: p
- **target**: blancura

## Root folder and read env variables

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Optimization-Industrial-Process


In [2]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN TRAINING

In [3]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import gcsfs
import pickle

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

# transform
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

# models
from sklearn.linear_model import LinearRegression # lr
from sklearn.linear_model import Ridge # ridge
from sklearn.linear_model import Lasso # lasso
from sklearn.tree import DecisionTreeRegressor # tree
from sklearn.ensemble import GradientBoostingRegressor #gb
from sklearn.ensemble import RandomForestRegressor #rf
#from xgboost import XGBRegressor # xgb
from  sklearn.neural_network import MLPRegressor # mlp

In [4]:
### desarrollo

PROJECT_ID = PROJECT_GCP
! gcloud config set project $PROJECT_ID

Updated property [core/project].


### 1. Read data

In [5]:
path_data = 'artifacts/data/data.pkl'
data = pd.read_pickle(path_data)
data.head()

,230AIT446.PNT,240AIC022.MEAS,240AIC126.MEAS,240AIC224.MEAS,240AIC286.MEAS,240AIC324.MEAS,240AIC433.MEAS,240AIT063A.PNT,240AIT063B.PNT,240AIT225A.PNT,...,S240ALDP022,S240ALDP031,S240ALDP032,S276PER002,S2MAQUINAT07,S76ALE017,SSTRIPPING015,calc_prod_d0,calc_prod_d1,calc_prod_p
datetime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:05:00,11.555040,2.983948,11.346645,4.413519,4.352375,10.441675,4.292521,5.869320,62.37495,1.837519,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3240.8635,3313.6215,3259.3745
2021-01-01 00:10:00,11.552320,3.015669,11.353215,4.413179,4.347186,10.432170,4.289684,5.869320,62.37495,1.814020,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3260.7475,3301.6920,3208.6785
2021-01-01 00:15:00,11.549955,3.018903,11.355525,4.408321,4.355828,10.410115,4.284427,5.869320,62.37495,1.814020,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3265.5765,3284.1330,3210.7790
2021-01-01 00:20:00,11.547145,3.001164,11.326725,4.408659,4.361292,10.379145,4.285478,5.835750,62.37495,1.814020,...,91.49,1.7,11.3,11.77,1.5712,173.6,964.0,3253.7750,3271.9260,3221.7745
2021-01-01 00:25:00,11.543160,3.017393,11.336345,4.408596,4.356374,10.387205,4.304148,5.802179,62.37495,1.814020,...,91.49,1.6,11.2,11.77,1.5712,173.6,964.0,3236.9790,3267.3050,3227.6935


### 2. Define target
All the stages could have multiple targets. But with the models supported by gurobi only accept models that predict one target

In [6]:
list_target = ['240AIT416B.PNT'] #blancura_salida_p

### 3. Define features
In the exploratory data analysis you should select the features that will be use to predict the target. In this example, this features are defined manually in a list

In stage D1, there are 2 VC - features controlables
- peroxido p
- soda p
- acido p (obs see the correlation, this feature has no correlation but it is used in the costs of p stage)

See the list of features controlables is a subset of list features controlables

In [7]:
list_features = [
    "240AIT322B.PNT", #brillo_p
    "calc_prod_p", #calc_prod_p
    "240FY312.RO01", # especifico_soda_p - VC
    "240FY397.RO01", #especifico_peroxido_p - VC
    "240AIC324.MEAS", #ph_p,
    "240FY430.RO01", #especifico_acido_p - VC
]

In [8]:
# list_features_controlables = [    
#     "240FY312.RO01", # especifico_soda_p - VC
#     "240FY397.RO01", #especifico_peroxido_p - VC
#     "240FY430.RO01", #especifico_acido_p - VC
# ]

### 4. Read master tags data for this stage. Sort features used to train according this order

In [9]:
### read master table - list tags
stage = 'p'
path_maestro_tags_d0eop = f'config/config_ml_models_development/MaestroTags-{stage}-general.xlsx'
maestro_tags = pd.read_excel(path_maestro_tags_d0eop)
maestro_tags

,TAG,TAG_DESCRIPTION,DESCRIPCION,ETAPA,CLASIFICACION,USE_PREVIOUS_MODEL,USE_NEXT_MODEL
0,240FI020A.PNT,prod_total,Producción Total,A,NC,NaN,NaN
1,calc_prod_p,calc_prod_p,Producción entrada P (prod entrada A desplazada),P,NC,NaN,NaN
2,240AIT322A.PNT,kappa_p,Kppa salida D1 (entrada P),P,NC,R - D1,NaN
3,240AIT322B.PNT,brillo_p,Brillo salida D1 (entrada P),P,NC,R - D1,NaN
4,240AIC324.MEAS,ph_p,pH entrada P,P,NC,R - D1,NaN
5,240TIT323.PNT,temperatura_p,T° entrada P,P,NC,NaN,NaN
6,240FY397.RO01,especifico_peroxido_p,Esp Peróxido P,P,C,NaN,NaN
7,240FY312.RO01,especifico_soda_p,Esp Soda,P,C,NaN,NaN
8,240FY430.RO01,especifico_acido_p,Esp Acido tac blanca,P,C,NaN,NaN
9,240AIT416B.PNT,blancura_salida_p,Blancura Salida P linea,TAC,R,NaN,NaN


In [10]:
### sort list of features according the order in master table

list_features = [tag for tag in maestro_tags['TAG'].tolist() if tag in list_features]
#list_features_controlables = [tag for tag in maestro_tags['TAG'].tolist() if tag in list_features_controlables]

### 5. Split train test

In [11]:
# RANDOM split train test
X_train, X_test, y_train, y_test = train_test_split(data[list_features], 
                                                    data[list_target], 
                                                    test_size = 0.2, 
                                                    random_state=42
                                                   )

In [12]:
# # TIME SERIES SPLIT
# X_train, X_test, y_train, y_test = train_test_split(data[list_features], 
#                                                     data[list_target], 
#                                                     test_size = 0.2, 
#                                                     shuffle = False
#                                                    )

In [13]:
### save data TRAIN - TEST
name_model = 'p_blancura'


# ---
# save X_train
path_X_train = f'artifacts/data_training/{name_model}/X_train.pkl'
with open(path_X_train, "wb") as output:
    pickle.dump(X_train, output)
    output.close()

# save y_train
path_y_train = f'artifacts/data_training/{name_model}/y_train.pkl'
with open(path_y_train, "wb") as output:
    pickle.dump(y_train, output)
    output.close()


# ---
# save X_test
path_X_test = f'artifacts/data_training/{name_model}/X_test.pkl'
with open(path_X_test, "wb") as output:
    pickle.dump(X_test, output)
    output.close()

# save y_test
path_y_test = f'artifacts/data_training/{name_model}/y_test.pkl'
with open(path_y_test, "wb") as output:
    pickle.dump(y_test, output)
    output.close()

### 6. Train lr and evaluate R2

In [14]:
# train lr
lr_model = LinearRegression()

lr = Pipeline([
    ('poly_feature_2', PolynomialFeatures(2)),
    #('scaler', MinMaxScaler() ), 
    ('lr',  lr_model)
])

lr.fit(X_train, y_train)

Pipeline(steps=[('poly_feature_2', PolynomialFeatures()),
                ('lr', LinearRegression())])

In [15]:
# r2 score
r2_lr_train = lr.score(X_train, y_train)
r2_lr_test = lr.score(X_test, y_test)

print('r2_train: ', r2_lr_train)
print('r2_test: ', r2_lr_test)

r2_train:  0.5323005031942587
r2_test:  0.5261727654287156


## SAVE OUTPUTS TRAINING

Al terminar el entrenamiento, los siguientes outputs deben de ser generados:

----
#### Artefacto Analitico:
- **modelo entrenado** y guardado como pkl

----
#### Listado de features:
- **listado de features** (listado de todas las features que ve el modelo)

- **listado de features variables controlables** (listado de todas las features que ve el modelo y que son variables controlables y por lo tanto
variables de decisión en un modelo de optimización)

- **listado de target** (lista con el target del modelo)


----
#### Example Input:
- **X_train.head(1)**: se necesita saber el orden de las features utilizadas y los nombres de las columnas. Ambos se deben de respetar. Con el listado de features se debe de poder deducir, pero de todas formas se guarda un ejemplo de la instancia de entrenamiento X

In [16]:
name_model = 'p_blancura'

### 1. Save artifact model

In [17]:
# save model
path_model_lr = f'artifacts/models/{name_model}/lr.pkl'
with open(path_model_lr, "wb") as output:
    pickle.dump(lr, output)
    output.close()

### 2. Save list of features
Save table master tag only with the tags used to train the model. 

OBS IMPORTANT: remember that the list of features was sorted according the master table so this order was used to train. Also this table has the differentation between no-controlable, controlable and targer variables

In [18]:
# generate a list of features + target
list_features_target = list_features + list_target

# filter master tag with only the features+target used to train the ml models
maestro_tags = maestro_tags[maestro_tags['TAG'].isin(list_features_target)]
maestro_tags = maestro_tags.reset_index().drop(columns = 'index')

In [19]:
# save master in config folder that will used to create the optimization engine
path_list_features_target_to_optimization = f'config/optimization_engine/ml_models/MaestroTags-{name_model}-general.xlsx'
maestro_tags.to_excel(path_list_features_target_to_optimization, index = False)

### 3. Save example input

In [20]:
# example input
example_input = X_train.head(1)
example_input

,calc_prod_p,240AIT322B.PNT,240AIC324.MEAS,240FY397.RO01,240FY312.RO01,240FY430.RO01
datetime,,,,,,
2022-05-19 06:10:00,3561.821,89.22453,10.43819,0.733446,2.816266,2.022609


In [21]:
# save example input
path_example_input_ml_model = f'config/optimization_engine/ml_models/{name_model}-example-input-model.xlsx'
example_input.to_excel(path_example_input_ml_model)